In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.formula.api import ols


# generate our student grade data
df = pd.DataFrame(np.array([[100, 96], [93, 90], [84, 89], [80, 85], [76, 80], [70, 68], [79, 75]]), columns = ['x', 'y'])

x = df['x']
y = df['y']

df.head(3)

,x,y
0,100,96
1,93,90
2,84,89


### Establish a Baseline

In [2]:
plt.rc("axes.spines", top=False, right=False)

In [3]:
# prediction
df['yhat_bl'] = df['y'].mean()
df.head(3)

,x,y,yhat_bl
0,100,96,83.285714
1,93,90,83.285714
2,84,89,83.285714


### Build a Simple Model

In [4]:
# generate parameters, i.e. create model
ols_model = ols('y ~ x', data=df).fit()

# compute predictions and add to original dataframe
df['yhat'] = ols_model.predict(x)

df.head()

,x,y,yhat_bl,yhat
0,100,96,83.285714,97.635214
1,93,90,83.285714,91.676524
2,84,89,83.285714,84.015350
3,80,85,83.285714,80.610384
4,76,80,83.285714,77.205418


### Evaluate Part 1: RMSE

Now we need to measure the performance of the baseline and the line created from our model. We need to determine if our model is "good enough". To determine this, we have to answer two questions.

#### Manually Compute Evaluation Metrics.

In this lesson, we will manually compute two common evaluation metric for regression models, the Mean Squared Error (MSE) and the Root Mean Squared Error (RMSE). To do so, we take the following steps:

1. Compute the residual, or error, for each data point.
2. Compute the SSE, Sum of Squared Errors, a.k.a. RSS, Residual Sum of Squares. This is simply squaring each of the errors computed in step one and summing them all together.
3. Compute the MSE, Mean Squared Error. We arrive at this by dividing your SSE by the total number of data points, i.e. the average of your errors that have each been squared.
4. Compute the RMSE, Root Mean Squared Error. Simply take the square root of the MSE.


In [5]:
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [8]:
# compute residuals
df['residual'] = df['yhat'] - df['y']
df['residual_bl'] = df['yhat_bl'] - df['y']
df.head()

,x,y,yhat_bl,yhat,residual,residual_bl
0,100,96,83.285714,97.635214,1.635214,-12.714286
1,93,90,83.285714,91.676524,1.676524,-6.714286
2,84,89,83.285714,84.015350,-4.984650,-5.714286
3,80,85,83.285714,80.610384,-4.389616,-1.714286
4,76,80,83.285714,77.205418,-2.794582,3.285714


$$SSE = \sum_{i=1}^{n}(\hat{y}-y_{i})^2$$

In [10]:
# square each residual value
df['residual^2'] = df.residual ** 2

df['residual_bl^2'] = df.residual_bl ** 2

df.head(3)

,x,y,yhat_bl,yhat,residual,residual_bl,residual^2,residual_bl^2
0,100,96,83.285714,97.635214,1.635214,-12.714286,2.673926,161.653061
1,93,90,83.285714,91.676524,1.676524,-6.714286,2.810732,45.081633
2,84,89,83.285714,84.015350,-4.984650,-5.714286,24.846737,32.653061


In [12]:
# SSE
SSE = sum(df['residual^2'])
SSE_baseline = sum(df['residual_bl^2'])

print("SSE = ", SSE)
print("SSE - baseline = ", SSE_baseline)

SSE =  96.85259593679459
SSE - baseline =  555.4285714285714


$$MSE = \frac{1}{n}\sum_{i=1}^{n} (\hat{y}-y_{i})^2$$

In [13]:
MSE = SSE/len(df)
MSE_baseline = SSE_baseline/len(df)

print("MSE = ", MSE)
print("MSE baseline = ", MSE_baseline)

MSE =  13.836085133827797
MSE baseline =  79.34693877551021


$$RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n} (\hat{y}-y_{i})^2}$$

In [14]:
from math import sqrt
RMSE = sqrt(MSE)
RMSE_baseline = sqrt(MSE_baseline)

print("RMSE = ", RMSE)
print("RMSE baseline = ", RMSE_baseline)

RMSE =  3.7196888490608724
RMSE baseline =  8.907689867497083


## Scikit-Learn
- Use scikit-learn to compute evaluation metrics

- Now, instead of manually computing the SSE, MSE, and RMSE, we will use sklearn.metrics to do so.

$$MSE = \frac{1}{n}\sum_{i=1}^{n} (\hat{y}-y_{i})^2$$
$$n*MSE = \sum_{i=1}^{n} (\hat{y}-y_{i})^2 = SSE$$
$$\therefore SSE = n*MSE$$

### SSE
- To compute the SSE or RSS, we must use the mean_squared_error function, as sklearn.metrics does not have a function for SSE directly. All we need to do is multiply the MSE by n, the number of data points.

In [18]:
SSE2 = mean_squared_error(df.y, df.yhat)*len(df)
SSE2_baseline = mean_squared_error(df.y, df.yhat_bl)*len(df)

print("SSE manual == SSE sklearn: ", SSE == SSE2) 
print("SSE manual - baseline == SSE sklearn - baseline: ", SSE_baseline == SSE2_baseline) 

SSE manual == SSE sklearn:  True
SSE manual - baseline == SSE sklearn - baseline:  True


### MSE
- Now, instead of manually computing the MSE, we will use sklearn.metrics.mean_squared_error to compute it.

In [17]:
MSE2 = mean_squared_error(df.y, df.yhat)

MSE2_baseline = mean_squared_error(df.y, df.yhat_bl)


print("MSE manual == MSE sklearn: ", MSE == MSE2) 
print("MSE manual - baseline == MSE sklearn - baseline: ", MSE_baseline == MSE2_baseline)

MSE manual == MSE sklearn:  True
MSE manual - baseline == MSE sklearn - baseline:  True


### RMSE
- Now, instead of manually computing the RMSE, we will use sklearn.metrics.mean_squared_error to compute it.

- We have to use mean_squared_error because there is not a function for RMSE. But that's easy because RMSE is only the square root of MSE.

In [20]:
RMSE2 = sqrt(mean_squared_error(df.y, df.yhat))
RMSE2_baseline = sqrt(mean_squared_error(df.y, df.yhat_bl))

print("RMSE manual == RMSE skearn: ", RMSE == RMSE2) 
print("RMSE manual - baseline == RMSE skearn - baseline: ", RMSE_baseline == RMSE2_baseline)

RMSE manual == RMSE skearn:  True
RMSE manual - baseline == RMSE skearn - baseline:  True


In [21]:
df_eval = pd.DataFrame(np.array(['SSE','MSE','RMSE']), columns=['metric'])
df_baseline_eval = pd.DataFrame(np.array(['SSE_baseline','MSE_baseline','RMSE_baseline']), columns=['metric'])

df_eval['model_error'] = np.array([SSE, MSE, RMSE])
df_baseline_eval['model_error'] = np.array([SSE_baseline, MSE_baseline, RMSE_baseline])

print(df_eval)
print(df_baseline_eval)

  metric  model_error
0    SSE    96.852596
1    MSE    13.836085
2   RMSE     3.719689
          metric  model_error
0   SSE_baseline   555.428571
1   MSE_baseline    79.346939
2  RMSE_baseline     8.907690


### Draw Conclusions
- Now, we will use our results to select the best model.

- We will compare each baseline metric with the respective metrics of the linear regression model to see if our model performs better.

In [22]:
df_eval['error_delta'] = df_eval.model_error - df_baseline_eval.model_error
df_eval

,metric,model_error,error_delta
0,SSE,96.852596,-458.575975
1,MSE,13.836085,-65.510854
2,RMSE,3.719689,-5.188001


In [23]:
yhat = df.yhat

## Evaluate Part 2: Model Significance

### Scikit-Learn
- Now we will compute R^2 using sklearn.metrics.explained_variance_scoore, which, as your remember, the coefficient of determination, R^2 is essentially the explained variance by its basic definition.

- You can also get the explained variance, or R^2 by using statsmodels.ols

In [24]:
# statsmodels.ols

r2 = ols_model.rsquared
print('R-squared = ', round(r2,3))

R-squared =  0.826


In [25]:
# sklearn.metrics.explained_variance_score

evs = explained_variance_score(df.y, df.yhat)
print('Explained Variance = ', round(evs,3))

Explained Variance =  0.826


In [27]:
# we care about R-squared and Prob (F-statistic):
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     23.67
Date:                Thu, 02 Apr 2020   Prob (F-statistic):            0.00461
Time:                        14:51:09   Log-Likelihood:                -19.128
No. Observations:                   7   AIC:                             42.26
Df Residuals:                       5   BIC:                             42.15
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.5111     14.641      0.855      0.432     -25.124      50.146
x              0.8512      0.175      4.866      0.005       0.402       1.301
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   0.983
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.776
Skew:                           0.124   Prob(JB):                        0.678
Kurtosis:                       1.388   Cond. No.                         737.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Draw Conclusions
We will now compute the p-value to determine if our test is significant, i.e. can we trust that the explained variance means what we think it means? If we have a high 
R
2
, does it really mean that there is correlation? If we have only two datapoints, then no! And our p-value should indicate that by giving us a high value. With 2 data points, we should have a VERY high probability of seeing an 
R
2
 of 1, but that does not indicate that the two variables are correlated.

In [28]:
f_pval = ols_model.f_pvalue

print("p-value for model significance = ", round(f_pval,4))

p-value for model significance =  0.0046


- If less than 0.05, you're OK => conclude that your regression model fits the data better than the model with no independent variables, meaning the independent variables in your model improve the fit.
- If greater than 0.05, it's probably better to stop using this set of features.

## Evaluate Part 3: Feature Significance

### T-test
#### The t-test for feature significance

- The null hypothesis states that the model without this variable fits the data as well as your model. (Significance > 0.05)

- The alternative hypothesis says that your model fits the data better with that independent variable than the model without that variable (Significance F <= 0.05)

In [29]:
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.826
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     23.67
Date:                Thu, 02 Apr 2020   Prob (F-statistic):            0.00461
Time:                        14:55:20   Log-Likelihood:                -19.128
No. Observations:                   7   AIC:                             42.26
Df Residuals:                       5   BIC:                             42.15
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.5111     14.641      0.855      0.432     -25.124      50.146
x              0.8512      0.175      4.866      0.005       0.402       1.301
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   0.983
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.776
Skew:                           0.124   Prob(JB):                        0.678
Kurtosis:                       1.388   Cond. No.                         737.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""